In [2]:
from url import *
from googlesearch import search

ModuleNotFoundError: No module named 'googlesearch'

In [ ]:
def fetch_ingredient_quantity(ingredients, ingredient):
    for i in ingredients:
        if ingredient in i['name']:
            return i['quantity'], i['unit']
    return 0, None

In [3]:
[i for i in search("How to bake", stop=1)][0]

'https://www.bakerssecret.com/blogs/food-articles/the-beginner-s-guide-to-baking-from-scratch?srsltid=AfmBOop59mmxXWENie_t5-iqFf74UmKxtSJBaP7mrdZDzuafD_9NicvS'

In [ ]:
text = input("Hello! What recipe would you like to cook today? Please paste the link here to get started.")
ingredients, instructions = fetch_page_from_url(text)
step = 0

while text != "Done":
    text = input("\nWhat would you like to do next?\n")
    
    if text == "What are the ingredients?":
        for i in print_ingredients_list(ingredients):
            print(i)
    
    elif text == "What is the next step?":
        if step >= len(instructions):
            print("That was the last step!")
        else:
            print(instructions[step])
            step += 1
    
    elif text == "What is the previous step?":
        print(instructions[step-1])
        
    elif text == "How do I do that?":
        # Need to create a better query somehow
        print("Please look at the following link for reference:")
        print([i for i in search("How to:" + instructions[step-1], stop=1)][0])
        
    # How much X do I need?
    
    # How many X do I need?
    
    else:
        print("I'm sorry, I'm not smart enough to understand that!")

ValueError: too many values to unpack (expected 2)

In [46]:
from url import *
import spacy
import nltk
from nltk.corpus import wordnet as wn
import re

In [47]:
nlp = spacy.load("en_core_web_md")

In [48]:
url = "https://www.allrecipes.com/recipe/218091/classic-and-simple-meat-lasagna/"

In [49]:
ingredients, instructions = fetch_page_from_url(url)

In [50]:
def extract_tools(instructions):
    # Initialize an empty set to collect tools
    tools_set = set()
    
    # Define a regex pattern to match common kitchen tool terms
    tool_patterns = r"\b(?:oven|pot|skillet|bowl|pan|foil|sheet|knife|dish|grater|plate|whisk|rack)\b"
    
    # Iterate over each instruction
    for instruction in instructions:
        # Process the instruction with SpaCy NLP pipeline
        doc = nlp(instruction)
        
        # Check each noun chunk to see if it matches the tool pattern
        for chunk in doc.noun_chunks:
            chunk_text = chunk.text.lower()
            # If the chunk matches the tool pattern, add it to the tools set
            if re.search(tool_patterns, chunk_text):
                tools_set.add(chunk_text)
            # Additional check with word embeddings to capture similar items
            elif any(token.similarity(nlp("kitchen tool")) > 0.5 for token in chunk):
                tools_set.add(chunk_text)
    
    return list(tools_set)

In [51]:
extract_tools(instructions)

/var/folders/rj/9zp9yl3s1jv50v3hrp7zghcm0000gn/T/ipykernel_2105/31509845.py:20: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  elif any(token.similarity(nlp("kitchen tool")) > 0.5 for token in chunk):


['the preheated oven',
 'a 9x13-inch baking pan',
 'foil',
 'a large skillet',
 'the dish',
 'a large bowl',
 'the oven',
 'a large pot',
 'aluminum foil']

In [52]:
def extract_tools(instructions):
    tools_set = set()
    
    # Define a regex pattern to match common kitchen tool terms
    tool_patterns = r"\b(?:oven|pot|skillet|bowl|pan|foil|sheet|knife|dish|grater|plate|whisk|rack)\b"
    
    # Reference phrase for tool similarity
    kitchen_tool_ref = nlp("kitchen tool")
    
    for instruction in instructions:
        doc = nlp(instruction)
        
        for chunk in doc.noun_chunks:
            chunk_text = chunk.text.lower()
            
            # First, check if chunk matches the tool pattern
            if re.search(tool_patterns, chunk_text):
                tools_set.add(chunk_text)
            else:
                # Perform document-level similarity if chunk has vectors
                if chunk.vector_norm and kitchen_tool_ref.vector_norm:  # Ensure both have vectors
                    similarity = chunk.similarity(kitchen_tool_ref)
                    if similarity > 0.5:
                        tools_set.add(chunk_text)
    
    return list(tools_set)

In [53]:
extract_tools(instructions)

['the preheated oven',
 'a 9x13-inch baking pan',
 'foil',
 'a large skillet',
 'the dish',
 'a large bowl',
 'the oven',
 'a large pot',
 'aluminum foil']